# TensorBoard 사용하기

### Load the TensorBoard notebook extension

In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime
import shutil, os
from tensorflow.keras.callbacks import TensorBoard

### Clear any logs from previous runs

In [3]:
if os.path.isdir('.\\logs\\'):
    shutil.rmtree('.\\logs\\')

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [5]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

try:
    if not(os.path.isdir('logs\\fit\\')):
        os.makedirs(os.path.join('logs\\fit\\'))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise

log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 9s 152us/sample - loss: 0.2185 - accuracy: 0.9352 - val_loss: 0.1097 - val_accuracy: 0.9660
Epoch 2/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0944 - accuracy: 0.9709 - val_loss: 0.0837 - val_accuracy: 0.9745
Epoch 3/5
60000/60000 [==============================] - 4s 69us/sample - loss: 0.0683 - accuracy: 0.9786 - val_loss: 0.0708 - val_accuracy: 0.9777
Epoch 4/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0526 - accuracy: 0.9833 - val_loss: 0.0655 - val_accuracy: 0.9812
Epoch 5/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0440 - accuracy: 0.9857 - val_loss: 0.0623 - val_accuracy: 0.9805


In [9]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 17264), started 0:06:28 ago. (Use '!kill 17264' to kill it.)

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [11]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [12]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [13]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [14]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [15]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.24751016497612, Accuracy: 92.74666595458984, Test Loss: 0.10995254665613174, Test Accuracy: 96.72000122070312
Epoch 2, Loss: 0.10618048161268234, Accuracy: 96.79000091552734, Test Loss: 0.08164308965206146, Test Accuracy: 97.47000122070312
Epoch 3, Loss: 0.07113699615001678, Accuracy: 97.86333465576172, Test Loss: 0.0644250214099884, Test Accuracy: 97.97999572753906
Epoch 4, Loss: 0.05331747978925705, Accuracy: 98.34832763671875, Test Loss: 0.06585239619016647, Test Accuracy: 97.9700012

In [17]:
%tensorboard --logdir logs/gradient_tape

Reusing TensorBoard on port 6006 (pid 17560), started 0:02:25 ago. (Use '!kill 17560' to kill it.)